In [26]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [27]:
%sql duckdb:///local.db


In [28]:
%%sql
INSTALL psql FROM community;
INSTALL spatial;
LOAD spatial;
LOAD psql;


Running query in 'duckdb:///local.db'

Success


In [29]:
%%sql

CREATE TABLE point_data AS
SELECT *
FROM ST_Read("osm21_cemetery.geojson")

Running query in 'duckdb:///local.db'

RuntimeError: (duckdb.duckdb.CatalogException) Catalog Error: Table with name "point_data" already exists!
[SQL: CREATE TABLE point_data AS
SELECT *
FROM ST_Read("osm21_cemetery.geojson")]
(Background on this error at: https://sqlalche.me/e/20/f405)


In [43]:
%%sql
CREATE OR REPLACE TABLE point_data_centroid AS
WITH centroid AS (
    FROM point_data
    SELECT centroid: geom.ST_Centroid(),
)
FROM centroid
SELECT
    id: ROW_NUMBER() OVER (ORDER BY centroid),
    locationX: centroid.ST_X(),
    locationy: centroid.ST_Y(),
    description: substr(md5(random()::VARCHAR), 1, 10)


Running query in 'duckdb:///local.db'

Count


In [44]:
%sql SELECT * FROM point_data_centroid

Running query in 'duckdb:///local.db'

id,locationX,locationy,description
1,71.60207635350525,54.78643014239917,8635a3cf61
2,68.07547055464238,40.23959944486414,a38db8bb2a
3,22.71972581720911,48.451325419849496,09ee56c7cb
4,29.491294654319063,46.07657549119183,d712c4bd92
5,79.38921734504402,52.74566659600047,6e0c10f6e4
6,6.359217615158059,43.142364551113644,c1bc6b682a
7,1.9459106666618595,49.15726517327247,9260faec9d
8,7.758160430065118,43.808794503076086,9d017e98ff
9,-0.9758063970803335,51.474531240130844,8acf845e2f
10,66.95163331926233,39.698323306344484,005bd82f5d


In [45]:
%%sql
SELECT COUNT(*) FROM point_data_centroid as data;

Running query in 'duckdb:///local.db'

count_star()
402325


In [46]:
%sql COPY point_data_centroid TO 'point_data_centroid.csv' (HEADER, DELIMITER ',');

Running query in 'duckdb:///local.db'

Count
